# Felipe Castillo
# Data Preparation
# Milestone 2
# 1/18/2022

In [44]:
import requests
import wikipedia as wp
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json
import os
import time


os.chdir("C:\DataScience_DSC_540\MilestoneFolder")



# Setting class for api to have instance of a particul year

Step 1: First I have to create a class to instantiate on a per year bases on the API. This will allow me to have more control on what type of information I want to pull back. Currently I only have campaign information I am after.

In [42]:
#https://www.propublica.org/datastore/api/propublica-congress-api


#At the moment only campagin finances are included can change
class CongressInformation:
    def __init__(self, year):
        self.year = year
        api_key = 'm7hiVghot77xDEUAUd3RIoW4FlLGgrSNBSAxgvlJ'
        self.headers = {'X-API-Key': api_key}

    def Propublica_Url(self):
        return "https://api.propublica.org/campaign-finance/v1/"+str(self.year)+"/candidates/leaders/pac-total.json"


    def JsonData(self, url):
        return requests.get(url=url, headers= self.headers).json()

Step 2: I have to capture that data inside a function. This function will take a year and return a json object. The function also connects to the congress class.

In [107]:
#connceting to class 
#instance is for year
def connect_finical_records_congress(year):
    try:
        congress_api = CongressInformation(year)
        time.sleep(1)# slowing down by one second allowing for first connect
        propublica_url = congress_api.Propublica_Url()
        json_object = congress_api.JsonData(propublica_url) 
        return json_object
    except:
        print("Could not connect to api.")




The json object is returned. However, its unreadable and needs to be reformatted. What is shown is the instance of 2016 congress records.

In [190]:
#returns a json object in a unreadable format
# needing main data like name, and party contributions
#setting a function to connect to the instance of a year
# more years can be look up an merged

#print(connect_finical_records_congress(2016))

# Formatting json object with function json mapper function

Step 3:The json object is nested and very hard to read. Plus there is alot of information in the data which is not needed. Once I extract the needed it will be in a array of congress records.

In [166]:
#Taking the json object and formatting it by the values
# Object come in by year 
# Data in a nested JSON reponse taking, main information by results
# Returning an array containing information needed
#Will take yearly data and insert it in the array congress is accessiable to other functions 

congress_data_array = []
def json_mapper(json_payload):
    
  
    date = json_payload['cycle']
    for results in json_payload['results']:
        name = results['name']
        #----------Step 4: State come in as /races/CO.json, which is incorrect and should be CO, for state.
        state = results['state']
        #data converstaion needs to be replaced only state intail required
        #/races/CO.json
        #need to by CO
        #checking if null
        if state is not None:
            temp_value = state.split("/")[-1].split('.')[0]
            state = temp_value  
        total_pac = results['total_from_pacs']
        total_ind = results['total_from_individuals']
        total_con = results['total_contributions']
        begin_cash = results['begin_cash']
        end_cash = results['end_cash']
        total_refunds = results['total_refunds']
        debts_owed = results['debts_owed']
        congress_data_array.append((name,
                                   state, 
                                   total_pac,
                                   total_ind, 
                                   total_con,
                                   begin_cash,
                                   end_cash, 
                                   total_refunds,
                                   debts_owed,
                                   date))
   
        
    
    

# Adding a readable header for all values

Step 5: Adding a header to the new data set. Currently the data set is only and array. This array will be built to map out the congress array data set that is stored from the api calls.

In [101]:
#making the headers for returned values 

header_api_congress = ['candidate_name',
                       'state',
                       'total_pac_money',
                       'total_individual', 
                       'total_contributions',
                       'begin_cash',
                       'end_cash', 
                       'total_refunds',
                       'debts_owed',
                       'date']

In [189]:
#connecting all known years to work in api
congress_years = ['2014','2012', '2006','2004','1998','1996','2018','2020']


#Taking in all years
for year in congress_years:
    #building congress class by year
    temp_year = connect_finical_records_congress(year)
    #mapping it with the function
    json_mapper(temp_year)
    







# Convert to more readable panda data frame


Step 6: As you can see the data is more readable now. However, it will be better saved in a data frame. Header and congress array both hold data. Now they just is need to be passed  into a table. 

In [162]:
#Has all testable years for campaign information
df_congress_api = pd.DataFrame(congress_data_array, columns=header_api_congress)
df_congress_api.head(10)

#Api information is now in a readable format

,candidate_name,state,total_pac_money,total_individual,total_contributions,begin_cash,end_cash,total_refunds,debts_owed,date
0,"MCCONNELL, MITCH",KY,3806660.00,12150309.00,17508470.00,7383877.0,252704.00,264879.0,336468.0,2014
1,"LANDRIEU, MARY L",LA,3372829.00,10236297.00,13659528.00,2534428.0,782616.00,83556.0,NaN,2014
2,"CORNYN, JOHN",TX,3348163.00,7907629.00,11255822.00,2671655.0,2954684.00,128722.0,NaN,2014
3,"PRYOR, MARK LUNSFORD",AR,3293383.00,8000604.00,11339394.00,1734136.0,239255.00,78005.0,NaN,2014
4,"ROBERTS, PAT",KS,3085837.00,3367728.00,6463693.00,886894.0,81281.00,24850.0,240408.0,2014
5,"COCHRAN, THAD",MS,3080820.00,3537276.00,6618097.00,480184.0,54954.00,7103.0,487200.0,2014
6,"HAGAN, KAY R",NC,2900412.00,16993199.00,19939016.00,1365627.0,188511.00,164792.0,NaN,2014
7,"MCCARTHY, KEVIN",CA,2848162.32,1521226.55,4369388.87,1824221.6,1988259.09,68300.0,10000.0,2014
8,"UDALL, MARK E",CO,2842094.00,13414189.00,16264228.00,1358707.0,336554.00,254365.0,NaN,2014
9,"WARNER, MARK ROBERT",VA,2811309.00,9556425.00,12431010.00,3704309.0,312737.00,93912.0,NaN,2014


# Formating wikipedia page by column with table 

Step 7: Now I am extracing data from wikipedia and not the api. There is one table in particular that I am looking for. I get the table then bulit it into a data frame.

In [7]:


def wikipedia_representatives():
    wiki_url = 'https://en.wikipedia.org/wiki/List_of_current_members_of_the_United_States_House_of_Representatives'
    
    try:
        response = requests.get(wiki_url)
        #using wikipedia packe to pull table
        html = wp.page('List of current members of the United States House of Representatives').html().encode("UTF-8")
        
        #pull exact table with congress information
        df_html = pd.read_html(html)[6]

        df_wiki = pd.DataFrame(df_html)
        
        return df_wiki

    except:
        print("Could note retieve website.")

In [165]:
wiki_df = wikipedia_representatives()


In [167]:
wiki_df.head()

,District,Member,Party,Party.1,Prior experience,Education,Assumed office,Residence,Born[2]
0,Alabama 1,Jerry Carl,NaN,Republican,Mobile County Commission,Florida Gateway College,2021,Mobile,(age 63)
1,Alabama 2,Barry Moore,NaN,Republican,Alabama House of Representatives,Enterprise State Community College (AS)Auburn ...,2021,Enterprise,(age 55)
2,Alabama 3,Mike Rogers,NaN,Republican,Calhoun County CommissionerAlabama House of Re...,"Jacksonville State University (BA, MPA)Birming...",2003,Anniston[3],(age 63)
3,Alabama 4,Robert Aderholt,NaN,Republican,Haleyville Municipal Judge,University of North AlabamaBirminghamâSouthe...,1997,Haleyville,(age 56)
4,Alabama 5,Mo Brooks,NaN,Republican,Alabama House of RepresentativesMadison County...,Duke University (BA)University of Alabama (JD),2011,Huntsville,(age 67)


# Populating last file source CSV 

In [184]:
senate_76_20_df = pd.read_csv("1976-2020-senate.csv")

In [185]:
senate_76_20_df.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party_detailed,writein,mode,candidatevotes,totalvotes,unofficial,version,party_simplified
0,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,SAM STEIGER,REPUBLICAN,False,total,321236,741210,False,20210114,REPUBLICAN
1,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,WM. MATHEWS FEIGHAN,INDEPENDENT,False,total,1565,741210,False,20210114,OTHER
2,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,DENNIS DECONCINI,DEMOCRAT,False,total,400334,741210,False,20210114,DEMOCRAT
3,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,ALLAN NORWITZ,LIBERTARIAN,False,total,7310,741210,False,20210114,LIBERTARIAN
4,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,BOB FIELD,INDEPENDENT,False,total,10765,741210,False,20210114,OTHER


Step 8: There was a few confusing column names, went a head and updated the column names. Then inserting them back into the orginal data drame.

In [186]:
#Updating some header names 
header_name = ['year','state','state_intails','state_fips','state_cen','state_ic','office','district','stage','special','candidate','political_parties','writein','mode','candidatevotes','totalvotesstate','unofficial','version','party_simplified']

senate_76_20_df.columns = [x for x in header_name]


In [187]:
senate_76_20_df.head()

,year,state,state_intails,state_fips,state_cen,state_ic,office,district,stage,special,candidate,political_parties,writein,mode,candidatevotes,totalvotesstate,unofficial,version,party_simplified
0,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,SAM STEIGER,REPUBLICAN,False,total,321236,741210,False,20210114,REPUBLICAN
1,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,WM. MATHEWS FEIGHAN,INDEPENDENT,False,total,1565,741210,False,20210114,OTHER
2,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,DENNIS DECONCINI,DEMOCRAT,False,total,400334,741210,False,20210114,DEMOCRAT
3,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,ALLAN NORWITZ,LIBERTARIAN,False,total,7310,741210,False,20210114,LIBERTARIAN
4,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,BOB FIELD,INDEPENDENT,False,total,10765,741210,False,20210114,OTHER
